In [86]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.utils import shuffle
import matplotlib.pyplot as plt

In [87]:
#import os
#from google.colab import drive
#drive.mount('/content/gdrive')

#path = "/content/gdrive/MyDrive/빅데이터 처리및 응용/1106_실습/Data"
#os.listdir(path)
path='.'

In [88]:
ratings = pd.read_csv(path + "/ratings_refined.csv", usecols=['userId', 'movieId', 'rating'])
ratings

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0
...,...,...,...
100784,610,166534,4.0
100785,610,168248,5.0
100786,610,168250,5.0
100787,610,168252,5.0


In [89]:
movies = pd.read_csv(path + "/movies_refined.csv", usecols=['movieId', 'title'])
movies = movies.set_index('movieId')
movies

,title
movieId,
1,Toy Story
2,Jumanji
3,Grumpier Old Men
4,Waiting to Exhale
5,Father of the Bride Part II
...,...
193581,Black Butler: Book of the Atlantic
193583,No Game No Life: Zero
193585,Flint


In [90]:
import pandas as pd
from scipy.spatial.distance import cosine

In [91]:
df = pd.merge(ratings, movies, on='movieId', how='left')
df

,userId,movieId,rating,title
0,1,1,4.0,Toy Story
1,1,3,4.0,Grumpier Old Men
2,1,6,4.0,Heat
3,1,47,5.0,Seven (a.k.a. Se7en)
4,1,50,5.0,"Usual Suspects, The"
...,...,...,...,...
100784,610,166534,4.0,Split
100785,610,168248,5.0,John Wick: Chapter Two
100786,610,168250,5.0,Get Out
100787,610,168252,5.0,Logan


In [92]:
df.columns[df.isna().any()].tolist()

[]

# Item-based CF

## 영화 유사도 행렬 준비

In [93]:
rating_matrix = df.pivot_table(values='rating', index='userId', columns='movieId')
rating_matrix

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,4.0,NaN,NaN,4.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,NaN,NaN,NaN,NaN,NaN,2.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
607,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
608,2.5,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 협업필터링

In [94]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

#정확도(MAE)를 계산하는 함수
def MAE(y_true, y_pred):
    return np.mean(np.abs(np.array(y_true) - np.array(y_pred)))
# 모델별 RMSE를 계산하는 함수
def score(model, neighbor_size=0):
    id_pairs = zip(x_test['userId'], x_test['movieId'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

def score1(model, neighbor_size=0):
    id_pairs = zip(x_test['userId'], x_test['movieId'])
    y_pred = np.array([model(user, movie, neighbor_size) for (user, movie) in id_pairs])
    y_true = np.array(x_test['rating'])
    return MAE(y_true, y_pred)

In [95]:
# train, test 데이터 분리
x = ratings.copy()
y = ratings['userId']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y, random_state=1)

# cosine 유사도

In [96]:
# 아이템 간 유사도 계산
rating_matrix_t = np.transpose(rating_matrix)

In [97]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=0)

imputed_data = imputer.fit_transform(rating_matrix_t)

matrix_dummy = pd.DataFrame(imputed_data, columns=rating_matrix_t.columns, index=rating_matrix_t.index)

C:\Users\kim sang soo\AppData\Roaming\Python\Python39\site-packages\sklearn\impute\_iterative.py:699: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [98]:
matrix_dummy

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.000000,3.794106,2.828692,4.409563,4.000000,3.732012,4.500000,4.370631,1.854954,4.143133,...,4.000000,3.771909,4.000000,3.000000,4.000000,2.500000,4.000000,2.500000,3.000000,5.000000
2,3.537885,2.674881,3.121529,2.692687,3.588900,4.000000,3.890606,4.000000,3.119612,2.918727,...,4.435901,4.000000,3.536176,5.000000,3.500000,3.110279,4.674964,2.000000,3.191916,3.879959
3,4.000000,1.583343,-0.107628,2.939754,3.429743,5.000000,3.285689,3.375410,1.630624,3.441387,...,3.919208,3.267692,3.754161,2.566919,2.837253,3.425168,4.631785,2.000000,3.178620,3.210043
4,4.529808,4.060063,2.506119,4.172783,3.300483,3.000000,3.141634,3.465800,2.515364,3.349467,...,3.919465,3.604080,3.200554,3.413399,3.057045,3.613452,3.728314,3.144324,3.164139,3.551001
5,4.277526,3.128680,3.640666,2.838420,4.121650,5.000000,1.931359,2.179644,1.551704,3.615764,...,4.021583,3.212704,2.023255,3.000000,3.133809,2.620363,3.436119,2.182167,3.126452,3.312852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,4.236548,3.955939,2.441248,3.541227,3.670649,3.425908,3.118883,3.227977,3.078546,3.324560,...,4.145548,3.307201,3.364904,3.397762,3.139567,3.556523,3.796980,3.129697,3.228289,3.521034
193583,4.228779,3.967854,2.436607,3.537588,3.612403,3.395619,3.142351,3.260564,2.941738,3.248333,...,4.140816,3.306986,3.345804,3.381332,3.180386,3.564913,3.773387,3.114013,3.223525,3.511159
193585,4.228779,3.967854,2.436607,3.537588,3.612403,3.395619,3.142351,3.260564,2.941738,3.248333,...,4.140816,3.306986,3.345804,3.381332,3.180386,3.564913,3.773387,3.114013,3.223525,3.511159


In [99]:
item_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
item_similarity = pd.DataFrame(item_similarity, index=rating_matrix_t.index, columns=rating_matrix_t.index)

In [100]:
item_similarity

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.937705,0.946105,0.966783,0.939480,0.950874,0.952176,0.963337,0.951073,0.935657,...,0.966850,0.966792,0.966751,0.966751,0.966850,0.966751,0.966850,0.966850,0.966850,0.967049
2,0.937705,1.000000,0.949577,0.964298,0.967719,0.938855,0.954739,0.966836,0.969262,0.922116,...,0.973491,0.972205,0.974617,0.974617,0.973491,0.974617,0.973491,0.973491,0.973491,0.972169
3,0.946105,0.949577,1.000000,0.962443,0.952227,0.950817,0.963910,0.964246,0.945058,0.937014,...,0.968604,0.968148,0.968902,0.968902,0.968604,0.968902,0.968604,0.968604,0.968604,0.968080
4,0.966783,0.964298,0.962443,1.000000,0.974405,0.972368,0.968391,0.991055,0.985837,0.961062,...,0.995555,0.995609,0.995340,0.995340,0.995555,0.995340,0.995555,0.995555,0.995555,0.995372
5,0.939480,0.967719,0.952227,0.974405,1.000000,0.957137,0.963671,0.977825,0.975664,0.937940,...,0.982298,0.981742,0.982694,0.982694,0.982298,0.982694,0.982298,0.982298,0.982298,0.981348
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.966751,0.974617,0.968902,0.995340,0.982694,0.974687,0.975252,0.993740,0.990851,0.963697,...,0.999919,0.999676,1.000000,1.000000,0.999919,1.000000,0.999919,0.999919,0.999919,0.999683
193583,0.966850,0.973491,0.968604,0.995555,0.982298,0.975133,0.974775,0.993804,0.990743,0.964314,...,1.000000,0.999919,0.999919,0.999919,1.000000,0.999919,1.000000,1.000000,1.000000,0.999765
193585,0.966850,0.973491,0.968604,0.995555,0.982298,0.975133,0.974775,0.993804,0.990743,0.964314,...,1.000000,0.999919,0.999919,0.999919,1.000000,0.999919,1.000000,1.000000,1.000000,0.999765


### Bias from mean

In [103]:
# 영화 평균 평점과 평점 편차 계산
item_rating_mean = rating_matrix.mean()
item_rating_bias = rating_matrix - item_rating_mean

In [104]:
# 아이템 기반 예측 함수
def IBCF_knn_bias_item_based(user_id, movie_id, neighbor_size=0):
    if user_id in item_rating_bias.index:
        # 현 영화와 다른 영화 간의 유사도 가져오기
        sim_scores = item_similarity[movie_id].copy()
        # 현 사용자의 평점 편차 가져오기
        user_ratings = item_rating_bias.loc[user_id].copy()
        # 현 사용자가 평가하지 않은 영화 삭제
        non_rating_idx = user_ratings[user_ratings.isnull()].index
        user_ratings = user_ratings.drop(non_rating_idx)
        sim_scores = sim_scores.drop(non_rating_idx)
        # Neighbor size 지정 여부에 따라 계산
        if neighbor_size > 0 and len(sim_scores) > 1:
            neighbor_size = min(neighbor_size, len(sim_scores))
            sim_scores = np.array(sim_scores)
            user_ratings = np.array(user_ratings)
            user_idx = np.argsort(sim_scores)
            sim_scores = sim_scores[user_idx][-neighbor_size:]
            user_ratings = user_ratings[user_idx][-neighbor_size:]
        prediction = np.dot(sim_scores, user_ratings) / sim_scores.sum()
        prediction = prediction + item_rating_mean[movie_id]
    else:
        prediction = item_rating_mean[movie_id]
    return prediction

In [105]:
neighbor_sizes = [10, 20, 30, 40, 50]
for size in neighbor_sizes:
    rmse = score(IBCF_knn_bias_item_based, size)
    mae=score1(IBCF_knn_bias_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")

Neighbor size: 10, RMSE: 0.5867138701351892, MAE: 0.4414567902177507

Neighbor size: 20, RMSE: 0.6369488997205782, MAE: 0.4805172798082168

Neighbor size: 30, RMSE: 0.664202053509504, MAE: 0.5011936544905357

Neighbor size: 40, RMSE: 0.6821387723266051, MAE: 0.5151881468198236

Neighbor size: 50, RMSE: 0.6945167873163363, MAE: 0.5250443260465674



In [106]:
neighbor_sizes = [0,2, 4, 6, 8]
for size in neighbor_sizes:
    rmse = score(IBCF_knn_bias_item_based, size)
    mae=score1(IBCF_knn_bias_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")

Neighbor size: 0, RMSE: 0.7890937300481577, MAE: 0.6015555102305485

Neighbor size: 2, RMSE: 0.3900587111141379, MAE: 0.2916978406293489

Neighbor size: 4, RMSE: 0.49922306416064466, MAE: 0.37490011916974597

Neighbor size: 6, RMSE: 0.5431220990045487, MAE: 0.4093239224371787

Neighbor size: 8, RMSE: 0.5689123650903559, MAE: 0.42871970325014835



In [107]:
def recommender_item_based(user, n_items=10):
    predictions = []
    for movie in rating_matrix.columns:
        predictions.append(IBCF_knn_bias_item_based(user, movie, neighbor_size=2))  # 인자 순서 수정
    recommendations = pd.Series(data=predictions, index=rating_matrix.columns, dtype=float)  # movieId를 인덱스로 사용
    recommendations = recommendations.sort_values(ascending=False)[:n_items]
    recommended_movies = movies.loc[recommendations.index]['title']  # 영화 ID에 해당하는 제목 추출
    return recommended_movies

In [108]:
# 영화 추천 함수 부르기
recommender_item_based(3, 10)

movieId
119145                         Kingsman: The Secret Service
1241                                 Dead Alive (Braindead)
7360                                       Dawn of the Dead
7991                                        Death Race 2000
2640                                               Superman
4446                      Final Fantasy: The Spirits Within
508                                            Philadelphia
3703                          Road Warrior, The (Mad Max 2)
2423      Christmas Vacation (National Lampoon's Christm...
22                                                  Copycat
Name: title, dtype: object

### KNN

In [109]:
def cf_knn_item_based(user_id, movie_id, neighbor_size=0):
    if user_id in rating_matrix:
        # 현재 영화와 다른 영화 간의 similarity 가져오기
        sim_scores = item_similarity[movie_id].copy()
        # 현재 사용자에 대한 모든 영화의 rating값 가져오기
        user_ratings = rating_matrix.loc[user_id].copy()
        # 현재 사용자가 평가하지 않은 영화의 index 가져오기
        none_rating_idx = user_ratings[user_ratings.isnull()].index
        # 현재 사용자가 평가하지 않은 영화의 rating (null) 제거
        user_ratings = user_ratings.drop(none_rating_idx)
        # 현재 사용자가 평가하지 않은 영화의 similarity값 제거
        sim_scores = sim_scores.drop(none_rating_idx)

        if neighbor_size == 0:
            # 모든 유사한 영화들의 가중평균값 구하기
            mean_rating = np.dot(sim_scores, user_ratings) / sim_scores.sum()
        else:
            if len(sim_scores) > 1:
                neighbor_size = min(neighbor_size, len(sim_scores))
                sim_scores = np.array(sim_scores)
                user_ratings = np.array(user_ratings)
                movie_idx = np.argsort(sim_scores)
                sim_scores = sim_scores[movie_idx][-neighbor_size:]
                user_ratings = user_ratings[movie_idx][-neighbor_size:]
                mean_rating = np.dot(sim_scores, user_ratings) / sim_scores.sum()
            else:
                mean_rating = 3.0
    else:
        mean_rating = 3.0
    return mean_rating

In [110]:
neighbor_sizes = [0,2, 4, 6, 8]
for size in neighbor_sizes:
    rmse = score(cf_knn_item_based, size)
    mae=score1(cf_knn_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")

Neighbor size: 0, RMSE: 0.9564395399058567, MAE: 0.7492068254328252

Neighbor size: 2, RMSE: 0.5345616147921843, MAE: 0.3323540890316016

Neighbor size: 4, RMSE: 0.5962978907995593, MAE: 0.4081071386182325

Neighbor size: 6, RMSE: 0.6262410655834153, MAE: 0.438305474939052

Neighbor size: 8, RMSE: 0.6456210262332516, MAE: 0.456991231333459



In [111]:
neighbor_sizes = [10,20, 30, 40, 50]
for size in neighbor_sizes:
    rmse = score(cf_knn_item_based, size)
    mae=score1(cf_knn_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")

Neighbor size: 10, RMSE: 0.6598470528550222, MAE: 0.4699511894114631

Neighbor size: 20, RMSE: 0.7069913428656588, MAE: 0.5123172876192211

Neighbor size: 30, RMSE: 0.7364629309200579, MAE: 0.538881262641172

Neighbor size: 40, RMSE: 0.7577858881605567, MAE: 0.5579961643676014

Neighbor size: 50, RMSE: 0.7737906183332006, MAE: 0.5727106317172397



In [112]:
# 아이템 기반 추천 함수
def recommender_item_based(user, n_items=10, neighbor_size=20):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index   # 이미 평가한 영화 확인
    items = rating_matrix.columns.difference(rated_index)  # 사용자가 평가하지 않은 영화
    for item in items:
        predictions.append(cf_knn_item_based(user, item, neighbor_size))       # 예상 평점 계산
    recommendations = pd.Series(data=predictions, index=items, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]  # 예상 평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

# 사용자에게 영화 추천
recommender_item_based(user=3, n_items=10, neighbor_size=2)

movieId
2122                              Children of the Corn
829                                    Joe's Apartment
2081                               Little Mermaid, The
1976                        Friday the 13th Part 3: 3D
79                                          Juror, The
1971    Nightmare on Elm Street 4: The Dream Master, A
3527                                          Predator
3258                                 Death Becomes Her
2174                                       Beetlejuice
3689                          Porky's II: The Next Day
Name: title, dtype: object

### Significance Weighting

In [113]:
# 사용자별 공통 평가 수 계산
rating_binary1 = np.array((rating_matrix > 0).astype(float))
rating_binary2 = rating_binary1.T
counts = np.dot(rating_binary2, rating_binary1)
counts = pd.DataFrame(counts, index=rating_matrix.columns, columns=rating_matrix.columns).fillna(0)


In [114]:
counts

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
movieId,,,,,,,,,,,,,,,,,,,,,
1,215.0,68.0,32.0,2.0,32.0,58.0,32.0,5.0,12.0,69.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,68.0,110.0,26.0,3.0,22.0,36.0,18.0,6.0,3.0,56.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,32.0,26.0,52.0,1.0,19.0,25.0,19.0,4.0,9.0,23.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2.0,3.0,1.0,7.0,3.0,2.0,6.0,1.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,32.0,22.0,19.0,3.0,49.0,24.0,23.0,4.0,9.0,20.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193583,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0
193585,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0


In [115]:
SIG_LEVEL = 3
MIN_RATINGS = 2

# train 데이터의 각 영화 평균과 사용자의 평점편차 계산
rating_mean = rating_matrix.mean(axis=0)
rating_bias = (rating_matrix - rating_mean).fillna(0)

In [116]:
def CF_knn_bias_sig_item_based(user_id, movie_id, neighbor_size=0):
    if user_id in rating_bias.columns:
        # 현 movie와 다른 영화 간의 유사도 가져오기
        sim_scores = item_similarity[movie_id]
        # 현 user의 평점편차 가져오기
        user_ratings = rating_bias.loc[user_id]
        # 현 user가 평가하지 않은 영화 표시
        non_rated = user_ratings.isnull()
        # 현 movie와 다른 영화 간 공통 평가 아이템 수 가져오기
        common_counts = counts[movie_id]
        # 공통으로 평가한 영화의 수가 SIG_LEVEL보다 낮은 영화 표시
        low_significance = common_counts < SIG_LEVEL
        # 평가를 안 하였거나, SIG_LEVEL이 기준 이하인 영화 제거
        non_rated_or_low_sig_idx = user_ratings[non_rated | low_significance].index
        user_ratings = user_ratings.drop(non_rated_or_low_sig_idx)
        sim_scores = sim_scores.drop(non_rated_or_low_sig_idx)
        # Neighbor size가 지정되지 않은 경우
        if neighbor_size == 0:
            # 편차로 예측값(편차 예측값) 계산
            prediction = np.dot(sim_scores, user_ratings) / sim_scores.sum()
            # 편차 예측값에 해당 영화의 평균 더하기
            prediction = prediction + rating_mean[movie_id]
        # Neighbor size가 지정된 경우
        else:
            # 해당 영화에 대해 평가한 사용자가 최소 MIN_RATINGS 이상인 경우에만 계산
            if len(sim_scores) > MIN_RATINGS:
                # 지정된 neighbor size 값과 해당 영화를 평가한 총 사용자 수 중 작은 것으로 결정
                neighbor_size = min(neighbor_size, len(sim_scores))
                # array로 바꾸기 (argsort를 사용하기 위함)
                sim_scores = np.array(sim_scores)
                user_ratings = np.array(user_ratings)
                # 유사도를 순서대로 정렬
                movie_idx = np.argsort(sim_scores)
                # 유사도와 rating을 neighbor size만큼 받기
                sim_scores = sim_scores[movie_idx][-neighbor_size:]
                user_ratings = user_ratings[movie_idx][-neighbor_size:]
                # 편차로 예측치 계산
                prediction = np.dot(sim_scores, user_ratings) / sim_scores.sum()+1
                # 예측값에 해당 영화의 평균 더하기
                prediction = prediction + rating_mean[movie_id]
            else:
                prediction = rating_mean[movie_id]
    else:
        prediction = rating_mean[movie_id]
    return prediction

In [117]:
neighbor_sizes = [0,2, 4, 6, 8]
for size in neighbor_sizes:
    rmse = score(CF_knn_bias_sig_item_based, size)
    mae = score1(CF_knn_bias_sig_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")


C:\Users\kim sang soo\AppData\Local\Temp\ipykernel_43596\904114406.py:20: RuntimeWarning: invalid value encountered in double_scalars
  prediction = np.dot(sim_scores, user_ratings) / sim_scores.sum()


Neighbor size: 0, RMSE: nan, MAE: nan

Neighbor size: 2, RMSE: 1.0315659875558338, MAE: 0.913334332429409

Neighbor size: 4, RMSE: 1.1068134291102176, MAE: 0.9221445191706773

Neighbor size: 6, RMSE: 1.138067832574371, MAE: 0.9296868041596553

Neighbor size: 8, RMSE: 1.1557122606509684, MAE: 0.9350266947288277



In [118]:
neighbor_sizes = [10,20, 30, 40, 50]
for size in neighbor_sizes:
    rmse = score(CF_knn_bias_sig_item_based, size)
    mae = score1(CF_knn_bias_sig_item_based, size)
    print(f"Neighbor size: {size}, RMSE: {rmse}, MAE: {mae}\n")


Neighbor size: 10, RMSE: 1.167590222513376, MAE: 0.9390858874861667

Neighbor size: 20, RMSE: 1.1929091429967742, MAE: 0.9474726700290433

Neighbor size: 30, RMSE: 1.202493659371332, MAE: 0.9511225233540637

Neighbor size: 40, RMSE: 1.2080991305981432, MAE: 0.953169397712791

Neighbor size: 50, RMSE: 1.211707961522006, MAE: 0.9546454168878835



In [119]:
# 추천하기
def recommender(user, n_items=10, neighbor_size=20):
    # 현재 사용자의 모든 아이템에 대한 예상 평점 계산
    predictions = []
    rated_index = rating_matrix.loc[user][rating_matrix.loc[user] > 0].index    # 이미 평가한 영화 확인
    items = rating_matrix.loc[user].drop(rated_index)
    for item in items.index:
        predictions.append(CF_knn_bias_sig_item_based(user, item, neighbor_size))  # 예상 평점 계산
    recommendations = pd.Series(data=predictions, index=items.index, dtype=float)
    recommendations = recommendations.sort_values(ascending=False)[:n_items]    # 예상 평점이 가장 높은 영화 선택
    recommended_items = movies.loc[recommendations.index]['title']
    return recommended_items

# 영화 추천 함수 부르기
recommender(3, 10, 2)

movieId
6460                                Trial, The (Procès, Le)
26810                                         Bad Boy Bubby
115122                            What We Do in the Shadows
174053                        Black Mirror: White Christmas
177593            Three Billboards Outside Ebbing, Missouri
3224                      Woman in the Dunes (Suna no onna)
7121                                             Adam's Rib
31364                   Memories of Murder (Salinui chueok)
2239      Swept Away (Travolti da un insolito destino ne...
1105                 Children of the Corn IV: The Gathering
Name: title, dtype: object